In [6]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import sqlite3


In [65]:
#conn = sqlite3.connect('work_mode.db')
#cursor = conn.cursor()

conn = sqlite3.connect('work_mode.db')
query = '''
SELECT client_id, time, latitude, longitude, altitude, speed, course, sat, name
FROM work_mode;
'''
df = pd.read_sql_query(query,conn)
df.shape


(159204, 9)

In [68]:
df['time'] = pd.to_datetime(df.time)
#df = df.iloc[1000:1100]
df = df.reset_index(drop=True)
df['speed'] = pd.to_numeric(df.speed)
df['latitude'] = pd.to_numeric(df.latitude)
df['longitude'] = pd.to_numeric(df.longitude)
#df_1 = df.groupby(by = ['client_id','time']).agg({'speed':'mean', 'latitude':'mean', 'longitude':'mean' })
df_1 = df
df = df.sort_values(by=['client_id', 'time'])
df_1['speed_mean'] = df_1.speed.rolling(window=5).mean()
df_1.shape

(159204, 10)

In [9]:
df['speed_mean'] = df.speed.rolling(window=5).mean()
df.head(20)

,client_id,time,latitude,longitude,altitude,speed,course,sat,name,speed_mean
26721,10,2020-08-09 15:08:00,55.561612,37.853608,154.8,48.9,268.7,33,,NaN
26722,10,2020-08-09 15:08:00,55.561608,37.853387,154.6,49.2,268.4,34,,NaN
26723,10,2020-08-09 15:08:00,55.561606,37.853164,154.5,49.4,269.0,33,,NaN
26725,10,2020-08-09 15:08:00,55.561603,37.852942,154.5,49.9,268.8,34,,NaN
26727,10,2020-08-09 15:08:00,55.561604,37.852716,154.6,50.3,270.4,33,,49.54
26729,10,2020-08-09 15:08:00,55.561607,37.852486,154.6,51.4,271.3,34,,50.04
26731,10,2020-08-09 15:08:00,55.561608,37.852254,154.6,52.3,270.5,34,,50.66
26733,10,2020-08-09 15:08:00,55.561608,37.852026,154.8,51.9,270.1,34,,51.16
26735,10,2020-08-09 15:08:00,55.561608,37.851798,155.0,51.3,269.9,34,,51.44
26737,10,2020-08-09 15:08:00,55.561606,37.851575,155.2,51.6,268.8,33,,51.70


In [85]:
X_2 = df_1.dropna()
X = X_2

In [220]:
X = StandardScaler().fit_transform(X)

{-1: 3, 0: 1146, 1: 163, 2: 84, 3: 10}

In [86]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=4, 
                       max_iter=100,
                    init_params='kmeans',
                    random_state=42)
X_2 = X_2.reset_index()
X_2 = X_2.set_index('time')
X_2 =X_2.drop(columns=['client_id','course', 'sat', 'name'])
X_2

,index,latitude,longitude,altitude,speed,speed_mean
time,,,,,,
2020-08-09 15:08:00,4,55.561604,37.852716,154.6,50.3,49.54
2020-08-09 15:08:00,5,55.561607,37.852486,154.6,51.4,50.04
2020-08-09 15:08:00,6,55.561608,37.852254,154.6,52.3,50.66
2020-08-09 15:08:00,7,55.561608,37.852026,154.8,51.9,51.16
2020-08-09 15:08:00,8,55.561608,37.851798,155.0,51.3,51.44
...,...,...,...,...,...,...
2020-10-09 17:11:00,159199,55.847844,37.638305,184.5,0.0,0.00
2020-10-09 17:11:00,159200,55.847844,37.638305,184.5,0.0,0.00
2020-10-09 17:11:00,159201,55.847844,37.638305,184.5,0.0,0.00


In [87]:
from sklearn.metrics import silhouette_score
gm.fit(X_2) #обучение модели
y_pred = gm.predict(X_2) # предсказание


# показываем, сколько кластеров и значений в них предсказано, -1 считаем выбросом
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{0: 43243, 1: 43955, 2: 35354, 3: 36648}

In [83]:
df_preg = pd.Series(data=y_pred, name='prediction')
                    
X_3 = pd.DataFrame({'y_pred':df_preg})  



X_2 = X_2.reset_index()
X_2['predict'] = X_3




In [77]:
X_4 = df_1.dropna()
X_4 = X_4.reset_index()
X_5 = X_4.client_id
X_5

0         10
1         10
2         10
3         10
4         10
          ..
159195    50
159196    50
159197    50
159198    50
159199    50
Name: client_id, Length: 159200, dtype: object

In [44]:
X_2 = X_2.reset_index()
X_2['client_id'] = X_5


In [48]:
X_2[X_2.client_id == '30']

,level_0,index,time,speed,latitude,longitude,speed_mean,predict,client_id
193,193,193,2020-07-08 04:37:00,23.382353,55.767732,37.598783,4.695944,3,30
194,194,194,2020-07-08 04:38:00,21.601667,55.766631,37.596415,9.016278,3,30
195,195,195,2020-07-08 04:39:00,46.920408,55.767737,37.594306,18.400359,3,30
196,196,196,2020-07-08 04:40:00,30.888333,55.772284,37.604272,24.578026,3,30
197,197,197,2020-07-08 04:41:00,75.138333,55.773507,37.619218,39.586219,3,30
...,...,...,...,...,...,...,...,...,...
419,419,419,2020-09-09 05:17:00,19.050000,55.741715,37.615839,21.486333,3,30
420,420,420,2020-09-09 05:18:00,71.753333,55.734052,37.613127,33.168667,3,30
421,421,421,2020-09-09 05:19:00,32.140351,55.730607,37.609056,35.402737,3,30
422,422,422,2020-09-09 05:20:00,29.244444,55.728802,37.618630,39.143626,3,30


In [84]:
silhouette_score(X=X_2, labels=y_pred, metric='euclidean')